In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from datetime import date

import json #for parsing the return from the Google API
import urllib #for passing info to the Google API
import requests
from bs4 import BeautifulSoup

To gather the data, we will scrape the below website.  The site has an embedded table for each year including all the games played in that year.  They also include the Associated Press poll ranking for each team for every game.  1950 is the first year where the AP poll ranked teams in the pre-season, giving teams ranks for their first games.

The goal here is to find the embedded table, extract it, save all the features as columns in a Pandas dataframe, combine all years, and save the final dataframe as a csv.

In [8]:
# test accessing the website
year = 2017
website = 'https://www.sports-reference.com/cfb/years/{}-schedule.html'.format(year)

response = requests.get(website)
soup = BeautifulSoup(response.text, 'html.parser')
print(response)

<Response [200]>


In [9]:
print(soup.prettify())

<!DOCTYPE html>
<html class="no-js" data-root="/home/cfb/build" data-version="klecko-" itemscope="" itemtype="https://schema.org/WebSite" lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="ie=edge" http-equiv="x-ua-compatible"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport">
   <link href="https://d2p3bygnnzw9w3.cloudfront.net/req/201910231" rel="dns-prefetch"/>
   <!-- no:cookie fast load the css.           -->
   <script>
    function gup(n) {n = n.replace(/[\[]/, '\\[').replace(/[\]]/, '\\]'); var r = new RegExp('[\\?&]'+n+'=([^&#]*)'); var re = r.exec(location.search);   return re === null?'':decodeURIComponent(re[1].replace(/\+/g,' '));}; document.srdev = gup('srdev')
   </script>
   <link crossorigin="" href="https://d2p3bygnnzw9w3.cloudfront.net" rel="preconnect"/>
   <link crossorigin="" href="https://d17lgqwvsissft.cloudfront.net" rel="preconnect"/>
   <style>
    html,body{margin:0;padding:0;font:14px/1.25 "Helvetica Neu

In [4]:
table = soup.find_all('table')
print(len(table[0].text))
print()
print((table[0].text))

85166

874 Games Table



Rk
Wk
Date
Time
Day
Winner
Pts

Loser
Pts
Notes



11Aug 26, 20173:00 PMSatBrigham Young20Portland State6LaVell Edwards Stadium - Provo, Utah
21Aug 26, 20172:30 PMSatColorado State58Oregon State27Sonny Lubrick Field at Colorado State Stadium - Fort Collins, Colorado
31Aug 26, 20176:00 PMSatHawaii38@Massachusetts35Warren McGuirk Alumni Stadium - Amherst, Massachusetts
41Aug 26, 20177:30 PMSat(19) South Florida42@San Jose State22CEFCU Stadium - San Jose, California
51Aug 26, 201710:00 PMSat(14) Stanford62Rice7Allianz Stadium - Sydney, AUS

Rk
Wk
Date
Time
Day
Winner
Pts

Loser
Pts
Notes

62Aug 31, 201710:30 PMThuArizona State37New Mexico State31Sun Devil Stadium - Tempe, Arizona
72Aug 31, 20178:00 PMThuArkansas49Florida A&M7War Memorial Stadium - Little Rock, Arkansas
82Aug 31, 20176:00 PMThuCentral Florida61Florida International17Spectrum Stadium - Orlando, Florida
92Aug 31, 20177:00 PMThuCentral Michigan30Rhode Island27Kelly/Shorts Stadium - Mount Pleasant, Mi

In [5]:
# We will use this function in the larger function below to parse beautiful soup to lists.
# This function has the added advantage of assigning None if the field does not exist, which
# happens in earlier years of the data.
def bs_to_list(bs):
    text_list = []
    if len (bs) > 0 :
        for item in bs:
            text_list.append(item.text)
        return text_list
    else:
        return None

In [6]:
# make a dataframe to hold all the data.  create df outside of loop.
df = pd.DataFrame()
for year in range (1950,2019,1):
    print(year, 'starting now...')

    # set the website we scarpe from using the year variable
    website = 'https://www.sports-reference.com/cfb/years/{}-schedule.html'.format(year)
    # get the response, parse it with bs4, and find the table tags
    response = requests.get(website)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find_all('table')
    
    # for all the desired data, find the associate data-stat tags
    row =  table[0].find_all('th', {'scope':'row'})
    week_number = table[0].find_all('td', {'data-stat':'week_number'})
    date_game = table[0].find_all('td', {'data-stat':'date_game'})
    time_game = table[0].find_all('td', {'data-stat':'time_game'})
    day_game =  table[0].find_all('td', {'data-stat':'day_name'})
    winner = table[0].find_all('td', {'data-stat':'winner_school_name'})
    winner_pts  = table[0].find_all('td', {'data-stat':'winner_points'})
    game_location = table[0].find_all('td', {'data-stat':'game_location'})
    loser = table[0].find_all('td', {'data-stat':'loser_school_name'})
    loser_pts = table[0].find_all('td', {'data-stat':'loser_points'})
    notes = table[0].find_all('td', {'data-stat':'notes'})
    
    # make an empty temp df and add each row as a columns.
    # this was neccessary to assign "None" to any fields that did not
    # exist in earlier years
    df_temp = pd.DataFrame()
    
    # iterate through the bs4
    df_temp['row'] = bs_to_list(row)
    df_temp['week_number'] = bs_to_list(week_number)
    df_temp['winner'] = bs_to_list(winner)
    df_temp['winner_pts'] = bs_to_list(winner_pts)
    df_temp['loser'] = bs_to_list(loser)
    df_temp['loser_pts'] = bs_to_list(loser_pts)
    df_temp['game_date'] = bs_to_list(date_game)
    df_temp['game_time'] = bs_to_list(time_game)
    df_temp['game_day'] = bs_to_list(day_game)
    df_temp['game_loc'] = bs_to_list(game_location)
    df_temp['notes'] = bs_to_list(notes)
    
    # assign a new column for each df_temp with the year
    df_temp['year'] = year
    # append the newly generated df_temp to the full df
    df = df.append(df_temp)
    print(year, 'complete!')
    # stop the scraper for a second to prevent overwhelming the website.
    # the terms of use for the site requested scraping no faster than a human 
    # could access the data and I figured this was a good time interval.
    time.sleep(5)

1950 starting now...
1950 complete!
1951 starting now...
1951 complete!
1952 starting now...
1952 complete!
1953 starting now...
1953 complete!
1954 starting now...
1954 complete!
1955 starting now...
1955 complete!
1956 starting now...
1956 complete!
1957 starting now...
1957 complete!
1958 starting now...
1958 complete!
1959 starting now...
1959 complete!
1960 starting now...
1960 complete!
1961 starting now...
1961 complete!
1962 starting now...
1962 complete!
1963 starting now...
1963 complete!
1964 starting now...
1964 complete!
1965 starting now...
1965 complete!
1966 starting now...
1966 complete!
1967 starting now...
1967 complete!
1968 starting now...
1968 complete!
1969 starting now...
1969 complete!
1970 starting now...
1970 complete!
1971 starting now...
1971 complete!
1972 starting now...
1972 complete!
1973 starting now...
1973 complete!
1974 starting now...
1974 complete!
1975 starting now...
1975 complete!
1976 starting now...
1976 complete!
1977 starting now...
1977 co

In [7]:
# Save the final file as a csv.  Append todays date to the filename
# to keep for overwriting the full file and to document when the site
# was scraped.
today = str(date.today())
df.to_csv('scraped_results/{}.csv'.format(today))